In [ ]:
IS_LINUX = False
from sys import platform
if platform == "linux" or platform == "linux2":
  IS_LINUX = True
if IS_LINUX:
    !pip install tensorflow-gpu
    !pip install jsonpickle

In [ ]:
if IS_LINUX:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

In [ ]:
if IS_LINUX:
    %cd /content/drive/My\ Drive/MastersThesis
    !ls
    !pwd
    import sys
    sys.path.append('./')

In [ ]:
import global_functions
global_functions.on_start()

In [ ]:
import tensorflow as tf
print(tf.__version__)
#2.0.0

In [ ]:
import experiments, datasets, training_data
import models
import constants
import vizualizer

In [ ]:
#basic_model = models.BasicModelProvider()(datasets.FashionMnistDataset())
trained_models.autoencoder.set_autoencoder_trainable(False)
existing_model =  models.ExistingModelProvider(lambda: trained_models)


In [ ]:
exp = experiments.ExperimentAutoencoder(datasets.MnistDataset, models.BasicModelProvider(),
                                                    training_data.BasicTrainingDataGeneratorAutoencoder())
result = exp.train(training_data.BasicTrainParametersAutoencoder(100, 128, 20, True, 1.0, 1e-5,
                                                                             None,
                                                                             False))

In [ ]:

exp = experiments.ExperimentClassifier(datasets.FashionMnistDataset, existing_model,
                                                   training_data.BasicTrainingDataGeneratorClassifier())
result = exp.train(training_data.BasicTrainParametersClassifier(100, 128, 20, True, 1.0, 1e-6, True,
                                                                            None,
                                                                            False))

In [ ]:
expe

In [18]:
import vizualizer, experiments , constants, importlib, datasets, models, predictors, training_data
importlib.reload(experiments)
importlib.reload(vizualizer)
importlib.reload(models)
importlib.reload(constants)
importlib.reload(predictors)
import numpy as np


results = experiments.ExperimentBase.load_experiment_results(constants.ExperimentsPaths.FashionMnist.CLASSIFIER, True)
experiment = results[-1].experiment
trained_models_classifier = experiment.model_provider()
results = experiments.ExperimentBase.load_experiment_results(constants.ExperimentsPaths.FashionMnist.AUTOENCODER, True)
experiment = results[-1].experiment
trained_models_autoencoder = experiment.model_provider()

#print(len(results))    
#vizualizer.Vizualizer.vizualize([result.train_history for result in results], 0.5)

In [ ]:
fashion_mnist = datasets.FashionMnistDataset()
predictions_autoencoder = experiments.ExperimentAutoencoder.predict_test(fashion_mnist ,trained_models)
experiments.ExperimentAutoencoder.evaluate_on_test(fashion_mnist, trained_models)
vizualizer.Vizualizer.show_random_autoencoder_images(fashion_mnist, fashion_mnist.get_test_images(), predictions_autoencoder, 5, 5)

In [ ]:
#trained_models = experiment.model_provider()
metrics = [ constants.Metrics.VAL_CLASSIFIER_OUT_ACCURACY, constants.Metrics.VAL_AUTOENCODER_OUT_LOSS]
params = vizualizer.VizualizeParams(0, 100, 5,False, False, metrics)
#vizualizer.Vizualizer.vizualize([result.train_history for result in results], params)
vizualizer.Vizualizer.vizualize([result.train_history for result in results[:]], params)

In [19]:
import numpy as np
fashion_mnist = datasets.FashionMnistDataset()
predictions_classifier = experiments.ExperimentClassifier.predict_test(fashion_mnist, trained_models_classifier)
predictions_classifier = np.argmax(predictions_classifier, axis = 1)
test_images = fashion_mnist.get_test_images()


10000/1 - 1s
Correct: 9387


In [23]:
incorrect_classifier = [i for i, value in enumerate(fashion_mnist.get_test_labels()) if predictions_classifier[i] != value]
correct_classifier = [i for i, value in enumerate(fashion_mnist.get_test_labels()) if predictions_classifier[i] == value]
len(incorrect_classifier)

613

In [24]:
#test_images[incorrect_classifier]
#trained_models_autoencoder.autoencoder.evaluate(test_images[incorrect_classifier],test_images[incorrect_classifier], verbose=2)
trained_models_autoencoder.autoencoder.evaluate(test_images[correct_classifier],test_images[correct_classifier], verbose=2)
#trained_models_autoencoder.autoencoder.evaluate(test_images,test_images, verbose=2)
#incorrect_classifier

9387/1 - 2s - loss: 0.2250


0.24337510873178736

In [ ]:
vizualizer.Vizualizer.show_autoencoder_images(datasets.FashionMnistDataset(), test_images, predictions_autoencoder, 5 )

In [ ]:
#ev = experiments.ExperimentAutoencoder.evaluate_on_train(datasets.MnistDataset() ,trained_models)
zoznam__fashion_mnist = []
test_images :datasets.Dataset = datasets.FashionMnistDataset().get_train_images()
for image in test_images:
    image = image.reshape(1, 28, 28, 1)
    evaluation = trained_models.autoencoder.evaluate(image, image, verbose = 0)
    zoznam__fashion_mnist.append(evaluation)
import numpy as np    


In [ ]:
np.mean(zoznam__fashion_mnist)

In [ ]:
last_layer = trained_models.classifier.layers[-1]
print(last_layer.activation)

In [ ]:
trained_models.make_classifier_without_activation()
#trained_models.make_classifier_with_sigmoid_activation()
#spomenut ho, ze bol - histogram sposobil, ze hodnoty boli viac na sebe a pre opacne datasety su podobne, neoddeli mi data. 
#trained_models.make_classifier_with_softmax_activation()

In [ ]:
predictors.Predictor.predict_for_subclasses(datasets.MnistDataset(),  datasets.FashionMnistDataset(), trained_models, 10)

In [ ]:
my_classes, distance = predictors.SquereDistanceFromMean().predict_test_based_on_train_treshoald( datasets.MnistDataset(), trained_models, 10.0)
#my_classes, distance = predictors.SquereDistanceFromMean().predict_train( datasets.FashionMnistDataset(), trained_models, None)
vizualizer.Vizualizer.plot_bins(distance, 10)

In [ ]:
import numpy as np
np.percentile(distance, 10)

In [ ]:
from abc import ABC, abstractmethod

class Predictor(ABC):
    def predict_train(self, dataset, model, threashold = None):
        return self.predict(dataset.get_train_images(), dataset.get_train_labels(), model, threashold)
        
    def predict_test(self, dataset, model, threashold = None):
        return self.predict(dataset.get_test_images(), dataset.get_test_labels(), model, threashold)
        
    def predict_test_based_on_train_treshoald(self, dataset, model, percentile):
        _, distance_train = self.predict_train(dataset, model)
        threashold_for_test =  np.percentile(distance_train, percentile)
        return self.predict_test(dataset, model, threashold_for_test), threashold_for_test
    
    @abstractmethod
    def predict(self, images, labels , model, threashold):
        pass
    

class AbsDistanceFromPredicted(Predictor):    
    
    def predict(self, images, labels , model, threashold):
        predictions = model.classifier.predict(images)
        my_classes = predictions.argmax(axis = 1)
        distances = np.zeros_like(my_classes)
        for index, _ in enumerate(predictions):
            classes_values = predictions[index]
            my_predicted_class = my_classes[index]
            distance = 0.0
            for index_class_value, class_value in enumerate(classes_values):
                if index_class_value == my_predicted_class:
                    continue
                distance += abs(classes_values[my_predicted_class] - class_value)
            if threashold is not None and distance < threashold:
                my_classes[index] = -1
            distances[index] = distance
        print("correct: ", (labels == my_classes).sum() ) 
        print("unknown: ", (-1 == my_classes).sum() )
        return my_classes, distances


    
class SquereDistanceFromPredicted(Predictor):
    
    def predict(self, images, labels , model, threashold):
        predictions = model.classifier.predict(images)
        my_classes = predictions.argmax(axis = 1)
        distances = np.zeros_like(my_classes)
        for index, _ in enumerate(predictions):
            classes_values = predictions[index]
            my_predicted_class = my_classes[index]
            distance = 0.0
            for index_class_value, class_value in enumerate(classes_values):
                if index_class_value == my_predicted_class:
                    continue
            
                distance += np.square(classes_values[my_predicted_class] - class_value)
            #distance /= (len(classes_values) - 1)
            if threashold is not None and distance < threashold:
                my_classes[index] = -1
            distances[index] = distance
        print("correct: ", (labels == my_classes).sum() ) 
        print("unknown: ", (-1 == my_classes).sum() )
        return my_classes, distances
    
class SumAbs(Predictor):

    def predict(self, images, labels , model, threashold):
        predictions = model.classifier.predict(images)
        my_classes = predictions.argmax(axis = 1)
        sums = np.zeros_like(my_classes)
        for index, _ in enumerate(predictions):        
            classes_values = predictions[index]
            sum_values = np.sum(np.absolute(classes_values)) 
            if threashold is not None and sum_values < threashold:
                my_classes[index] = -1
            sums[index] = sum_values
        print("correct: ", (labels == my_classes).sum() ) 
        print("unknown: ", (-1 == my_classes).sum() )
        return my_classes, sums

class SumSquere(Predictor):    
    
    def predict(self, images, labels , model, threashold):
        predictions = model.classifier.predict(images)
        my_classes = predictions.argmax(axis = 1)
        sums = np.zeros_like(my_classes)
        for index, _ in enumerate(predictions):        
            classes_values = predictions[index]
            sum_values = np.sum(np.square(classes_values)) 
            if threashold is not None and sum_values < threashold:
                my_classes[index] = -1
            sums[index] = sum_values
        print("correct: ", (labels == my_classes).sum() ) 
        print("unknown: ", (-1 == my_classes).sum() )
        return my_classes, sums


# na zaklade hodnoty maximalnej predikovanej hodnoty a thresholdu, ziadna suma ani vzdialenost
# 13.26
class MaxArgPredictor(Predictor):

    def predict(self, images, labels , model, threashold):
        predictions = model.classifier.predict(images)
        my_classes = predictions.argmax(axis = 1)
        sums = np.zeros_like(my_classes)
        for index, _ in enumerate(predictions):        
            class_value = predictions[index][my_classes[index]]
            if threashold is not None and class_value < threashold:
                my_classes[index] = -1
        print("correct: ", (labels == my_classes).sum() ) 
        print("unknown: ", (-1 == my_classes).sum() )
        predictions = np.max(predictions, axis = 1) 
        return my_classes, predictions
    

#my_classes, distances = predict_with_unknown_class_squere_distance_from_predicted(datasets.MnistDataset(), trained_models, 9108.0)
#my_classes, distances = predict_with_unknown_class_squere_distance_from_predicted(datasets.FashionMnistDataset(), trained_models, 9108.0)
    

In [ ]:
predictors = Predictor.__subclasses__()
threashoald_list = []
for predictor in predictors:
    print(predictor.__name__)
    instance = predictor()
    (_, __), threashoald = instance.predict_test_based_on_train_treshoald(datasets.MnistDataset(), trained_models, 10)
    threashoald_list.append(threashoald)

In [ ]:
for index, predictor in enumerate(predictors):
    print(predictor.__name__)
    instance = predictor()
    print("Threashold: ", threashoald_list[index])
    instance.predict_test(datasets.MnistDataset(), trained_models, threashoald_list[index])

In [ ]:
fashion_mnist =  datasets.MnistDataset()
fashion_mnist.get_test_labels()
correct_predict = np.argmax( predictions, axis = 1) == fashion_mnist.get_test_labels()
predictions = predictions[correct_predict]
predictions = np.max(predictions, axis = 1)

In [ ]:
def plot_bins(predictions, number_of_bins):
    import numpy as np
    import matplotlib.pyplot as plt
    bins = np.linspace( predictions.min() - 0.01 , predictions.max() + 0.01, num=number_of_bins)

    def bins_labels(bins, **kwargs):
        bin_w = (max(bins) - min(bins)) / (len(bins) - 1)
        plt.xticks(np.arange(min(bins)+bin_w/2, max(bins), bin_w), bins, **kwargs)
        plt.xlim(bins[0], bins[-1])

    print(bins)
    fig, ax = plt.subplots(1,1)
    hist = ax.hist(predictions , bins=bins)
    labels = [str(count) for count in hist[0]]
    #labels.insert(0, 0)
    print(labels)
    #ax.set_xticklabels(labels)
    plt.show()
    return bins, labels

mnist_bins, mnist_labels = plot_bins(distances, 20)
#fashion_mnist_bins, fashion_mnist_labels = plot_bins(fashion_mnist_predictions, 9)


In [ ]:
def print_intervals(bins, labels):
    for index, bin in enumerate(bins[:-1]):
        print("interval: <{:.4f} - {:.4f}, count:{})".format(bins[index] , bins[index  + 1], labels[index]))
#print_intervals(fashion_mnist_bins, fashion_mnist_labels)
print_intervals(mnist_bins, mnist_labels)

In [ ]:
import matplotlib.pyplot as plt
rng = np.random.RandomState(10)  # deterministic random data
a = np.hstack((rng.normal(size=1000), rng.normal(loc=5, scale=2, size=1000)))
_ = plt.hist(a, bins=[-5, 0, 5, 10])  # arguments are passed to np.histogram
plt.title("Histogram with 'auto' bins")
plt.show()

In [ ]:
# from keras.utils import plot_model
# keras.utils.plot_model(models.auto_classifier, show_shapes=True, to_file = 'model_combined.png')
import global_functions, constants, vizualizer, experiments
logs = global_functions.get_files_in_dir_with_extension(constants.Paths.OUTPUT_DIRECTORY, ".json" )


In [ ]:
import vizualizer, experiments
results = experiments.ExperimentBase.load_experiment_results(constants.ExperimentsPaths.FashionMnist.AUTOENCODER2_CLASSIFIER1_AUTO_L_OFF)
vizualizer.Vizualizer.vizualize([result.train_history for result in results], 0.5)

    

In [ ]:
model = results[-1].experiment.model_provider()
model.classifier .summary()

In [ ]:
import vizualizer
import importlib
importlib.reload(vizualizer)
vizualizer.Vizualizer.show_autoencoder_images(datasets.FashionMnistDataset(), data.test_y,  predictions, 5)

In [ ]:
def make_batches(to_batch, batch_size = 128):
    return [to_batch[i * batch_size:(i + 1) * batch_size] for i in range((len(to_batch) + batch_size - 1) // batch_size )]

batch_size = 128
to_batch = tf.data.Dataset.from_tensor_slices((train_images, keras.utils.to_categorical(train_labels, 10) ))
train_batched = to_batch.batch(batch_size)

In [ ]:
epochs = 10
for epoch in range(epochs):
    print("Epoch: {}", epoch)
    for step, (images, labels) in enumerate(train_batched):
        set_autoencoder_trainable(True)
        with tf.GradientTape() as tape:
          prediction = autoencoder(images)
          loss_autoencoder = autoencoder.loss(images, prediction)
          gradients = tape.gradient(loss_autoencoder, autoencoder.trainable_variables)
          autoencoder.optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables))  
        set_autoencoder_trainable(False)
        with tf.GradientTape() as tape:
          predict_class = classifier(images)
          loss_classifier = classifier.loss(labels, predict_class)  
#           classiefier_trainable = classifier_head.trainable_variables
          classiefier_trainable = classifier.trainable_variables
          gradients = tape.gradient(loss_classifier, classiefier_trainable)
          classifier.optimizer.apply_gradients(zip(gradients  ,classiefier_trainable) ) 
          if step % 10 == 0:
              print("Step: {}, loss autoencoder: {}, loss classifier: {}".format(step, loss_autoencoder.numpy().mean(), loss_classifier.numpy().mean()))


In [ ]:
for epoch in range(5):
  print("Classifier")
  historyClassifier = classifier.fit(x = train_images, y = train_labels_one_hot_encoding, batch_size=128, epochs=1, validation_data= (test_images, test_labels_one_hot_encoding))

In [ ]:
predictions_auto_encoder, predictions_classifier = auto_classifier.predict(test_images)
predictions_classifier = np.argmax(np.round(predictions_classifier),axis=1)
correct = np.where(predictions_classifier==test_labels)[0]
print( "Found {} correct labels".format(len(correct)))

In [ ]:

test_eval = classifier.evaluate(test_images, test_labels_one_hot_encoding, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])
predicted_classes = classifier.predict(test_images)
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)
predicted_classes = predicted_classes.astype('uint8')
correct = [predicted_classes[i] == value for i, value in enumerate(test_labels)]
print("Correct: {}".format( correct.count(True)))


